# Tracer Advection Stencil

#### 1. Start a parallel cluster

In [1]:
from functions import show_clusters

import ipyparallel as ipp
import time

cluster = ipp.Cluster(engines='mpi', n=6).start_and_connect_sync()


Starting 6 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/6 [00:00<?, ?engine/s]

In [3]:
time.sleep(10)
show_clusters()
%autopx

cluster_id        state    cluster_file
1657755950-lcs4  running   /home/ajdas/.ipython/profile_default/security/cluster-1657755950-lcs4.json
%autopx enabled


#### 2. Create a domain configuration

In [5]:
%%capture
from mpi4py import MPI
import functions as func


mpi_comm = MPI.COMM_WORLD
mpi_size = mpi_comm.Get_size()
mpi_rank = mpi_comm.Get_rank()

backend = 'numpy'
layout = (1, 1)
nx, ny, nz, nhalo = 100, 100, 79, 3
dimensions = {'nx': nx, 'ny': ny, 'nx1': nx+1, 'ny1': ny+1, 'nz': nz, 'nhalo': nhalo, 'tile': mpi_size, 'nxhalo': nx+2*nhalo, 'nyhalo': ny+2*nhalo}
units = {'dist': 'm', 'coord': 'degrees', 'qvapor': 'kg/kg', 'psi': 'kg/m/s', 'wind': 'm/s'}
origins = {'halo': (0, 0), 'compute_2d': (dimensions['nhalo'], dimensions['nhalo']), 'compute_3d': (dimensions['nhalo'], dimensions['nhalo'], 0)}


configuration = func.configure_domain(layout, mpi_comm, dimensions, backend=backend)

%px:   0%|          | 0/6 [00:00<?, ?tasks/s]

In [211]:
grid_type = 0
hord = 6
# ???????

#### 2a. Gather domain coordinates

In [11]:
from pace.util import Quantity
import numpy as np

dxa_halo =  Quantity(configuration['grid_data'].dxa.data, ('x_halo', 'y_halo'), units['dist'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
dya_halo =  Quantity(configuration['grid_data'].dya.data, ('x_halo', 'y_halo'), units['dist'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
dxa_global = configuration['communicator'].gather(dxa_halo)
dya_global = configuration['communicator'].gather(dya_halo)

lona_halo =  Quantity(configuration['grid_data'].lon_agrid.data * 180 / np.pi, ('x_halo', 'y_halo'), units['coord'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
lata_halo =  Quantity(configuration['grid_data'].lat_agrid.data * 180 / np.pi, ('x_halo', 'y_halo'), units['coord'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
lona_global = configuration['communicator'].gather(lona_halo)
lata_global = configuration['communicator'].gather(lata_halo)

#### 2b. Save coordinates to netCDF file

Convert below markdown cell to code.

```
import functions as func
dx_halo =  Quantity(configuration['grid_data'].dx.data, ('x_halo', 'y_halo'), units['dist'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
dy_halo =  Quantity(configuration['grid_data'].dy.data, ('x_halo', 'y_halo'), units['dist'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
dxc_halo =  Quantity(configuration['grid_data'].dxc.data, ('x_halo', 'y_halo'), units['dist'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
dyc_halo =  Quantity(configuration['grid_data'].dyc.data, ('x_halo', 'y_halo'), units['dist'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
lon_halo =  Quantity(configuration['grid_data'].lon.data * 180 / np.pi, ('x_halo', 'y_halo'), units['coord'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
lat_halo =  Quantity(configuration['grid_data'].lat.data * 180 / np.pi, ('x_halo', 'y_halo'), units['coord'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)

dx_global = configuration['communicator'].gather(dx_halo)
dy_global = configuration['communicator'].gather(dy_halo)
dxc_global = configuration['communicator'].gather(dxc_halo)
dyc_global = configuration['communicator'].gather(dyc_halo)
lon_global = configuration['communicator'].gather(lon_halo)
lat_global = configuration['communicator'].gather(lat_halo)

fOut = 'coordinates.nc'
variables = {'dx': dx_global, 'dy': dy_global, 
             'dxa': dxa_global, 'dya': dya_global, 
             'dxc': dxc_global, 'dyc': dyc_global, 
             'lon': lon_global, 'lat': lat_global, 
             'lona': lona_global, 'lata': lata_global}

if mpi_rank == 0:
    func.store_coordinates(fOut, dimensions, variables)
```

#### 3. Create initial conditions on a plane

#### 3a. Create water vapor initial condition

In [12]:
if mpi_rank == 0:
    gaussian_multiplier = func.create_gaussianMultiplier(np.deg2rad(lona_global.data), np.deg2rad(lata_global.data), dimensions, center_tile=0)
else:
    gaussian_multiplier = np.zeros((dimensions['tile'], dimensions['nxhalo'], dimensions['nyhalo']))

mpi_comm.Bcast(gaussian_multiplier, root=0)
qvapor = Quantity(gaussian_multiplier[mpi_rank], ('x', 'y'), units['qvapor'], origins['compute_2d'], (dimensions['nx'], dimensions['ny']), backend)
qvapor_global = configuration['communicator'].gather(qvapor)

[stdout:0] Centering gaussian on lon=351.01, lat=1.01


#### 3b. Run coordinates through streamfunction generation

In [13]:
psi, _ = func.calculate_streamfunction_testCase1(np.deg2rad(lona_halo.data), np.deg2rad(lata_halo.data), dimensions)

psi_halo = Quantity(psi, ('x_halo', 'y_halo'), units['psi'], origins['halo'], (dimensions['nxhalo'], dimensions['nyhalo']), backend)
psi = Quantity(psi, ('x', 'y'), units['psi'], origins['compute_2d'], (dimensions['nx'], dimensions['ny']), backend)
psi_global = configuration['communicator'].gather(psi)

#### 3c. Calculate winds from streamfunction

In [14]:
ua, va = func.calculate_windsFromStreamfunction(psi_halo, dxa_halo, dya_halo, dimensions)

ua = Quantity(ua, ('x', 'y'), units['wind'], origins['compute_2d'], (dimensions['nx'], dimensions['ny']), backend)
va = Quantity(va, ('x', 'y'), units['wind'], origins['compute_2d'], (dimensions['nx'], dimensions['ny']), backend)

ua_global = configuration['communicator'].gather(ua)
va_global = configuration['communicator'].gather(va)

#### 3d. Plot initial conditions and save data to file

In [20]:
%%capture capt
fOut = 'initialize_winds.nc'

lona =  Quantity(configuration['grid_data'].lon_agrid.data * 180 / np.pi, ('x_interface', 'y_interface'), units['coord'], origins['compute_2d'], (dimensions['nx1'], dimensions['ny1']), backend)
lata =  Quantity(configuration['grid_data'].lat_agrid.data * 180 / np.pi, ('x_interface', 'y_interface'), units['coord'], origins['compute_2d'], (dimensions['nx1'], dimensions['ny1']), backend)
lon_global = configuration['communicator'].gather(lona)
lat_global = configuration['communicator'].gather(lata)

if mpi_rank == 0:
    variables = {'lon': lon_global.data, 'lat': lat_global.data, 'ua': ua_global.data, 'va': va_global.data, 'psi': psi_global.data, 'qvapor': qvapor_global.data}

    func.plot_projection_field(variables['lon'], variables['lat'], variables['qvapor'], vmin=-0, vmax=1, cmap='viridis', title='q_vapor init', units=units['qvapor'])
    func.plot_projection_field(variables['lon'], variables['lat'], variables['ua'], vmin=-40, vmax=40, cmap='bwr', title='ua init', units=units['wind'])
    func.plot_projection_field(variables['lon'], variables['lat'], variables['va'], vmin=-40, vmax=40, cmap='bwr', title='va init', units=units['wind'])
    func.plot_projection_field(variables['lon'], variables['lat'], variables['psi'], vmin=-3e8, vmax=3e8, cmap='bwr', title='psi init', units=units['psi'])

    func.write_initialCondition_toFile(fOut, variables, dimensions, units)

%px:   0%|          | 0/6 [00:00<?, ?tasks/s]

In [ ]:
capt

#### 3e. Write initial conditions to file

In [15]:
#importlib.reload(func)


if mpi_rank == 0:

In [ ]:
# fInit = 'initialize_advection.nc'

# if mpi_rank == 0:
#     %run -i initialize_streamfunction_advection.py $fOut $fInit